In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from tqdm import tqdm
from webdriver_manager.firefox import GeckoDriverManager

In [3]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Driver [/home/victor/.wdm/drivers/geckodriver/linux64/v0.27.0/geckodriver] found in cache


In [4]:
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#cap = DesiredCapabilities().FIREFOX
#cap["marionette"] = False

In [5]:
#options = Options()
#options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
#driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Program Files (x86)\Google\Chrome\chromedriver.exe", )

In [6]:
#обозначили сайт, проверили, чтобы 200 ок
page_link = 'https://www.facebook.com/groups/realtor.russia'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 1319, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/home/victor/anaconda3/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/victor/anaconda3/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/victor/anaconda3/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/victor/anaconda3/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/home/victor/anaconda3/lib/python3.7/http/client.py", line 966, in send
    self.connect()
  File "/home/victor/anaconda3/lib/python3.7/ht

FakeUserAgentError: Maximum amount of retries reached

In [7]:
driver.get(page_link)

In [47]:
for i in range(25):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [60]:
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [61]:
soup

<html class="_9dls" dir="ltr" id="facebook" lang="ru"><head><link data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" rel="shortcut icon"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><meta content="Группы Facebook" name="application-name"/><meta content="https://www.facebook.com/groups/" name="application-url"/><meta content="Риэлторы: информация и аргументы для клиентов: 23 583 участника. Эта группа создана для обмена профессиональной информацией. Особенно приветствуются..." name="description"/><link href="https://www.facebook.com/groups/realtor.russia/" rel="canonical"/><link href="https://m.facebook.com/groups/realtor.russia/?locale2=ru_RU" media="only screen and (max-width: 640px)" rel="alternate"/><link href="https://m.facebook.com/groups/realtor.russia/?local

In [62]:
soup.findAll('div', attrs = {'class':"kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"})
#текст постов

[<div class="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"><div dir="auto" style="text-align: start;">Эта группа создана для обмена профессиональной информацией. Особенно приветствуются здесь ссылки на полезные для риэлтора ресурсы и на информацию, которая может содержать аргументы для работы с клиентами.</div></div>,
 <div class="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"><div dir="auto" style="text-align: start;">Бдительность риэлтора спасла семью москвичей от ушлого квартиранта . </div></div>,
 <div class="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"><div dir="auto" style="text-align: start;">Коллеги, а как вы относитесь видимо тоже к коллегам, рекламирующим и предлагающим объект от агенства, а когда приходишь в офис агенства на внесение аванса/задатка оказывается, что предлагают заключить соглашение с собственником, внести средства собственнику, получить расписку от собственника. И ни где нет следа об агенстве. То есть, если собственник откажется пров

In [63]:
b = soup.findAll('span', attrs = {'class':"gpro0wi8 j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb"})

In [64]:
a = soup.findAll('b', attrs = {'class':"b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4"})

In [68]:
parsed_ls = []
for i in range(len(b)):
    parsed = (b[i].text.replace('=','').replace('·','').replace('\xa0', ''))
    parsed_ls.append(parsed)

In [69]:
for i in range(len(parsed_ls)):
    parsed_ls[i] = parsed_ls[i].replace(' ', '')
    parsed_ls[i] = parsed_ls[i].replace('г.Москва', '')
    if 'ч.' in parsed_ls[i]:
        parsed_ls[i] = '14сентября2020'
    if 'мин.' in parsed_ls[i]:
        parsed_ls[i] = '14сентября2020'
    if 'Вчера' in parsed_ls[i]:
        parsed_ls[i] = '13сентября2020'
    if parsed_ls[i][:2].isnumeric():
        par

In [86]:
MonthDict[parsed_ls[1][2:-4]]

9

In [83]:
MonthDict={ "января" : 1,
      "февраля": 2,
       "марта": 3,
       "апреля": 4,
       "мая": 5,
       "июня": 6,
       "июля": 7,
       "августа": 8,
       "сентября": 9,
       "октября": 10,
       "ноября": 11,
       "декабря": 12
}

In [85]:
MonthDict['января']

1

In [82]:
parsed_ls

['13мая2014г.',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '10сентябряв08:17',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '11сентябряв09:09',
 '14сентября2020',
 '14сентября2020',
 '13сентября2020',
 '9сентябряв06:16',
 '12сентябряв06:32',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '12сентябряв04:21',
 '9сентябряв23:51',
 '13сентябряв01:11',
 '11сентябряв11:47',
 '14сентября2020',
 '13сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '11сентябряв13:48',
 '11сентябряв13:43',
 '14сентября2020',
 '14сентября2020',
 '12сентябряв23:21',
 '11сентябряв07:01',
 '11сентябряв13:49',
 '8сентябряв02:51',
 '12сентябряв07:57',
 '11сентябряв10:31',
 '14сентября2020',
 '12сентябряв00:36',
 '10сентябряв08:00',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '14сентября2020',
 '12сентябряв06:57',
 '12сентябряв06:28',
 '13сентябряв11: